Import the required modules

In [32]:
from pyspark.sql import SparkSession

Create a spark session

In [33]:
spark = SparkSession.builder.appName("reading_fro_postgreqls")\
        .config('spark.jars.packages', 'org.postgresql:postgresql:42.5.4,mysql:mysql-connector-java:8.0.26')\
        .getOrCreate()
sqlContext = SparkSession(spark)
spark.sparkContext.setLogLevel("ERROR")

In [34]:
spark

Extract data from the table(employees) in  postgresql.

In [35]:
postgre_df = spark.read.format("jdbc") \
    .option("url", "jdbc:postgresql://192.168.0.104:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "public.employees") \
    .option("user", "postgres") \
    .option("password", "erick")\
    .load()

In [36]:
postgre_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department_id: integer (nullable = true)



In [37]:
postgre_df.show()

+---+----------+---------+------+-------------+
| id|first_name|last_name|salary|department_id|
+---+----------+---------+------+-------------+
| 45|     Kevin|   Duncan| 45210|         1003|
| 25|    Pamela| Matthews| 57944|         1005|
| 48|    Robert|    Lynch|117960|         1004|
| 34|    Justin|     Dunn| 67992|         1003|
| 62|      Dale|    Hayes| 97662|         1005|
|  1|      Todd|   Wilson|110000|         1006|
| 61|      Ryan|    Brown|120000|         1003|
| 21|   Stephen|    Berry|123617|         1002|
| 13|     Julie|  Sanchez|210000|         1001|
| 55|   Michael|   Morris|106799|         1005|
| 44|    Trevor|   Carter| 38670|         1001|
| 73|   William|  Preston|155225|         1003|
| 39|     Linda|    Clark|186781|         1002|
| 10|      Sean| Crawford|190000|         1006|
| 30|   Stephen|    Smith|194791|         1001|
| 75|     Julia|    Ramos|105000|         1006|
| 59|     Kevin| Robinson|100924|         1005|
| 69|    Ernest| Peterson|115993|       

In [38]:
postgre_df.head()

Row(id=45, first_name='Kevin', last_name='Duncan', salary=45210, department_id=1003)

Get the maximum salary from each employee

In [39]:
max_salary=postgre_df.groupBy("id", "first_name", "last_name", "department_id")\
    .max("salary") \
    .withColumnRenamed("max(salary)", "max_salary").orderBy("id")

In [40]:
max_salary.show()

+---+----------+---------+-------------+----------+
| id|first_name|last_name|department_id|max_salary|
+---+----------+---------+-------------+----------+
|  1|      Todd|   Wilson|         1006|    110000|
|  2|    Justin|    Simon|         1005|    130000|
|  3|     Kelly|  Rosario|         1002|     42689|
|  4|  Patricia|   Powell|         1004|    170000|
|  5|    Sherry|   Golden|         1002|     44101|
|  6|   Natasha|  Swanson|         1005|     90000|
|  7|     Diane|   Gordon|         1002|     74591|
|  8|  Mercedes|Rodriguez|         1005|     61048|
|  9|   Christy| Mitchell|         1001|    150000|
| 10|      Sean| Crawford|         1006|    190000|
| 11|     Kevin| Townsend|         1002|    166861|
| 12|    Joshua|  Johnson|         1004|    123082|
| 13|     Julie|  Sanchez|         1001|    210000|
| 14|      John|  Coleman|         1001|    152434|
| 15|   Anthony|   Valdez|         1001|     96898|
| 16|    Briana|    Rivas|         1005|    151668|
| 17|     Ja

Get employees getting over 150000  salary

In [41]:
salary_above150k=max_salary.filter("max_salary >150000")

In [42]:
salary_above150k.show()

+---+----------+---------+-------------+----------+
| id|first_name|last_name|department_id|max_salary|
+---+----------+---------+-------------+----------+
|  4|  Patricia|   Powell|         1004|    170000|
| 10|      Sean| Crawford|         1006|    190000|
| 11|     Kevin| Townsend|         1002|    166861|
| 13|     Julie|  Sanchez|         1001|    210000|
| 14|      John|  Coleman|         1001|    152434|
| 16|    Briana|    Rivas|         1005|    151668|
| 22|  Brittany|    Scott|         1002|    162537|
| 30|   Stephen|    Smith|         1001|    194791|
| 39|     Linda|    Clark|         1002|    186781|
| 42|     Traci| Williams|         1003|    180000|
| 50|  Victoria|   Wilson|         1002|    176620|
| 52|      Kara|    Smith|         1004|    192838|
| 60|   Charles|  Pearson|         1004|    173317|
| 73|   William|  Preston|         1003|    155225|
| 74|   Richard|     Cole|         1003|    180361|
+---+----------+---------+-------------+----------+



Get the departments of employes with over 150000 and count the number of employees in those  departments.

In [46]:
department_info=salary_above150k.groupBy("department_id").count(). orderBy("count")

In [45]:
department_info.show()

+-------------+-----+
|department_id|count|
+-------------+-----+
|         1006|    1|
|         1005|    1|
|         1003|    3|
|         1001|    3|
|         1004|    3|
|         1002|    4|
+-------------+-----+



save the information in the mysql database.

In [31]:
department_info.write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", "jdbc:mysql://192.168.0.104:3306/erick") \
  .option("dbtable", "depertmentdata") \
  .option("user", "root") \
  .option("password", "erick") \
  .save()